In [10]:
# 1) Minimal Responses API (string input)

from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()  # <-- important

openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)  # or just OpenAI() if env var is set

response = client.responses.create(
    model="gpt-5-nano",
    input="What is 2+2?"
)

print(response.output_text)


In [9]:

# 2) Chat-style” messages (system + user), still using Responses
from openai import OpenAI
client = OpenAI()

messages = [
    {"role": "system", "content": "You are a concise tutor."},
    {"role": "user", "content": "What is 2+2?"}
]

response = client.responses.create(
    model="gpt-5-nano",
    input=messages
)

print(response.output_text)


In [12]:
# 3) Streaming (print tokens as they arrive)

from openai import OpenAI

client = OpenAI()

stream = client.responses.create(
    model="gpt-5-nano",
    input=[{"role": "user", "content": "Explain 2+2 in one sentence."}],
    stream=True,
)

for event in stream:
    if event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)

print()  # newline at end



In [11]:
# 4) Keep conversation context across calls

from openai import OpenAI
client = OpenAI()

r1 = client.responses.create(
    model="gpt-5-nano",
    input="Tell me a short joke."
)
print(r1.output_text)

r2 = client.responses.create(
    model="gpt-5-nano",
    previous_response_id=r1.id,
    input=[{"role": "user", "content": "Explain why it’s funny."}]
)
print(r2.output_text)


In [13]:
# 5) Structured Outputs (typed JSON) with Pydantic

from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class Answer(BaseModel):
    question: str
    result: int
    explanation: str

resp = client.responses.parse(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "Return structured data."},
        {"role": "user", "content": "Compute 2+2 and explain briefly."},
    ],
    text_format=Answer,
)

data: Answer = resp.output_parsed
print(data.result)
print(data.explanation)



In [14]:
# 6) Function calling (tool calling) in Responses (minimal end-to-end)

from openai import OpenAI
import json

client = OpenAI()

tools = [
    {
        "type": "function",
        "name": "add",
        "description": "Add two integers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "integer"},
                "b": {"type": "integer"},
            },
            "required": ["a", "b"],
        },
    }
]

def add(a: int, b: int) -> int:
    return a + b

input_list = [{"role": "user", "content": "Use the add tool to compute 2+2."}]

# 1) model suggests a tool call
r1 = client.responses.create(model="gpt-5", tools=tools, input=input_list)
input_list += r1.output

# 2) execute tool call(s), append outputs
for item in r1.output:
    if item.type == "function_call" and item.name == "add":
        args = json.loads(item.arguments)
        result = add(args["a"], args["b"])
        input_list.append({
            "type": "function_call_output",
            "call_id": item.call_id,
            "output": json.dumps({"result": result}),
        })

# 3) model produces final answer using tool result
r2 = client.responses.create(
    model="gpt-5-nano",
    tools=tools,
    input=input_list,
    instructions="Reply with the final numeric answer only."
)
print(r2.output_text)
